In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

file_path = r'C:\Users\BAE J.H\Desktop\우울증예측1.xlsx'
data = pd.read_excel(file_path)

train_data = data[(data['연도'] >= 2000) & (data['연도'] <= 2024)].copy()

X_train = train_data[['독거노인인구', '우울증환자수']]  
y_train = train_data['독거노인인구'] * (train_data['우울증환자수비율'] / 100)  

# 다중 선형 회귀 모델 사용
model = LinearRegression()
model.fit(X_train, y_train)

years_to_predict = np.arange(2025, 2071)
future_data = pd.DataFrame({
    '연도': years_to_predict,
    '독거노인인구': np.nan,  
    '우울증환자수': np.nan, 
    '우울증환자수비율': np.nan  
})

initial_population_growth_rate = 0.16  
initial_depression_growth_rate = 0.162  

population_growth_rate_increase = 0.002  
depression_growth_rate_increase = 0.0167 

max_population_growth_rate = 0.02 
max_depression_growth_rate = 0.001

last_known_year = 2024
last_known_population = data.loc[data['연도'] == last_known_year, '독거노인인구'].values[0]
last_known_depression_rate = data.loc[data['연도'] == last_known_year, '우울증환자수비율'].values[0]

for i, year in enumerate(years_to_predict):
    population_growth_rate = initial_population_growth_rate + population_growth_rate_increase * (year - 2001)
    depression_rate_growth_rate = initial_depression_growth_rate + depression_growth_rate_increase * (year - 2001)
    
    population_growth_rate = min(population_growth_rate, max_population_growth_rate)
    depression_rate_growth_rate = min(depression_rate_growth_rate, max_depression_growth_rate)

    future_data.loc[future_data['연도'] == year, '독거노인인구'] = last_known_population * (1 + population_growth_rate)
    future_data.loc[future_data['연도'] == year, '우울증환자수비율'] = last_known_depression_rate + depression_rate_growth_rate

    last_known_population = future_data.loc[future_data['연도'] == year, '독거노인인구'].values[0]
    last_known_depression_rate = future_data.loc[future_data['연도'] == year, '우울증환자수비율'].values[0]

future_data['우울증환자수'] = future_data['독거노인인구'] * (future_data['우울증환자수비율'] / 100)

# 다중 선형 회귀 모델을 사용해 예측
X_future = future_data[['독거노인인구', '우울증환자수']]  
future_data['독거노인우울증수'] = model.predict(X_future)

pd.set_option('display.float_format', '{:,.0f}'.format)

print(future_data[['연도', '독거노인우울증수']])

future_data.to_excel(r'C:\Users\BAE J.H\Desktop\예측_결과.xlsx', index=False)